# CSGY-6513 Big Data Homework 5

## Name: Runze Li

In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 9.77 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37637,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 9.77 GiB
Comm: tcp://127.0.0.1:43635,Total threads: 1
Dashboard: http://127.0.0.1:38699/status,Memory: 2.44 GiB
Nanny: tcp://127.0.0.1:46747,


## 1 Same as Homework 2, Question 7, In Dask - 25 points

### (a) Find food service and active restaurants closest to the following coordinate of [35.994914, -78.897133], and show the first 20.

In [2]:
import dask.dataframe as dd
import pandas as pd
import ast
from haversine import haversine, Unit

In [3]:
# Load data
file_path = "shared/data/Restaurants_in_Durham_County_NC.json"
df1 = pd.read_json(file_path)

In [4]:
# Normalize the 'fields' column to flatten its attributes
df1 = pd.json_normalize(df1['fields'])
df1.head()

,status,geolocation,premise_zip,rpt_area_desc,risk,est_group_desc,seats,water,premise_phone,premise_state,...,premise_address2,opening_date,premise_name,transitional_type_desc,smoking_allowed,id,sewage,premise_address1,hours_of_operation,closing_date
0,ACTIVE,"[35.9207272, -78.9573299]",27707,Food Service,4,Full-Service Restaurant,60.0,5 - Municipal/Community,(919) 403-0025,NC,...,SUITE 6C,1994-09-01,WEST 94TH ST PUB,FOOD,NO,56060,3 - Municipal/Community,4711 HOPE VALLEY RD,NaN,NaN
1,ACTIVE,"[36.0467802, -78.8895483]",27704,Food Service,4,Nursing Home,350.0,5 - Municipal/Community,(919) 479-9966,NC,...,NaN,2003-10-15,BROOKDALE DURHAM IFS,FOOD,NO,58123,3 - Municipal/Community,4434 BEN FRANKLIN BLVD,NaN,NaN
2,ACTIVE,"[35.9182655, -78.9593263]",27707,Food Service,2,Fast Food Restaurant,7.0,5 - Municipal/Community,(919) 489-7300,NC,...,NaN,2009-07-09,SMOOTHIE KING,FOOD,NO,70266,3 - Municipal/Community,1125 W. NC HWY 54 SUITE 806,NaN,NaN
3,ACTIVE,"[36.0183378, -78.9060312]",27701,Food Service,2,Full-Service Restaurant,100.0,5 - Municipal/Community,(919) 688-8880,NC,...,NaN,2012-01-09,HAMPTON INN & SUITES,FOOD,NO,97837,3 - Municipal/Community,1542 N GREGSON ST,NaN,NaN
4,ACTIVE,"[36.0556347, -78.9135175]",27704,Residential Care,0,NaN,6.0,5 - Municipal/Community,(919) 477-5825,NC,...,NaN,2008-06-02,BETTER LIVING CONCEPTS OF DURHAM,N/A,N/A,60690,3 - Municipal/Community,909 GARCIA ST,NaN,NaN


In [5]:
df1['latitude'] = df1['geolocation'].apply(lambda x: x[0] if isinstance(x, list) else None).astype(float)
df1['longitude'] = df1['geolocation'].apply(lambda x: x[1] if isinstance(x, list) else None).astype(float)

In [6]:
# Convert the filtered Pandas DataFrame to a Dask DataFrame
dask_df1 = dd.from_pandas(df1, npartitions=4)

In [7]:
dask_df1

,status,geolocation,premise_zip,rpt_area_desc,risk,est_group_desc,seats,water,premise_phone,premise_state,insp_freq,type_description,premise_city,premise_address2,opening_date,premise_name,transitional_type_desc,smoking_allowed,id,sewage,premise_address1,hours_of_operation,closing_date,latitude,longitude
npartitions=4,,,,,,,,,,,,,,,,,,,,,,,,,
0,string,string,string,string,int64,string,float64,string,string,string,int64,string,string,string,string,string,string,string,string,string,string,string,string,float64,float64
616,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
dask_df1 = dask_df1[(dask_df1['status'] == 'ACTIVE') & (dask_df1['rpt_area_desc'] == 'Food Service')]

In [9]:
coord1 = (35.994914, -78.897133)

In [10]:
dask_df1['distance'] = dask_df1.apply(
    lambda row: haversine((row['latitude'], row['longitude']), coord1),
    axis=1
)

/opt/conda/envs/bigdata-spark/lib/python3.11/site-packages/dask_expr/_collection.py:3231: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'float64'))

  warnings.warn(meta_warning(meta))


In [11]:
result1 = dask_df1.sort_values(by='distance')

In [12]:
print("First 20 food service and active restaurants closest to [35.994914, -78.897133]:")
result1.compute().head(20)

First 20 food service and active restaurants closest to [35.994914, -78.897133]:


,status,geolocation,premise_zip,rpt_area_desc,risk,est_group_desc,seats,water,premise_phone,premise_state,...,transitional_type_desc,smoking_allowed,id,sewage,premise_address1,hours_of_operation,closing_date,latitude,longitude,distance
1255,ACTIVE,"[35.9932826, -78.8981331]",27701,Food Service,4,Full-Service Restaurant,44.0,5 - Municipal/Community,(919) 667-9525,NC,...,FOOD,NO,85098,3 - Municipal/Community,310 E. MAIN ST.,<NA>,<NA>,35.993283,-78.898133,0.202491
1975,ACTIVE,"[35.9938322, -78.8993041]",27701,Food Service,2,Fast Food Restaurant,60.0,5 - Municipal/Community,(919) 560-0082,NC,...,FOOD,NO,57000,3 - Municipal/Community,200 E MAIN ST,<NA>,<NA>,35.993832,-78.899304,0.229393
969,ACTIVE,"[35.9958445, -78.8996129]",27701,Food Service,4,Full-Service Restaurant,105.0,5 - Municipal/Community,(919) 973-1589,NC,...,FOOD,N/A,152033,3 - Municipal/Community,102 CITY HALL PLAZA SUITE 101,<NA>,<NA>,35.995844,-78.899613,0.245926
2086,ACTIVE,"[35.994841, -78.89987]",27701,Food Service,2,<NA>,20.0,5 - Municipal/Community,(571) 276-9362,NC,...,FOOD,N/A,179705,3 - Municipal/Community,117 E MAIN ST,<NA>,<NA>,35.994841,-78.899870,0.246367
393,ACTIVE,"[35.9954939, -78.8997764]",27701,Food Service,2,Fast Food Restaurant,90.0,5 - Municipal/Community,(919) 680-2333,NC,...,FOOD,NO,85604,3 - Municipal/Community,107 E. PARRISH STREET,<NA>,<NA>,35.995494,-78.899776,0.246398
1645,ACTIVE,"[35.9959577, -78.8995561]",27701,Food Service,2,Full-Service Restaurant,36.0,5 - Municipal/Community,(919) 294-6661,NC,...,FOOD,N/A,179066,3 - Municipal/Community,104 CITY HALL PLAZA SUITE 101,<NA>,<NA>,35.995958,-78.899556,0.246959
880,ACTIVE,"[35.9952125, -78.8998725]",27701,Food Service,4,<NA>,30.0,5 - Municipal/Community,(206) 691-3640,NC,...,FOOD,N/A,173987,3 - Municipal/Community,110 E PARRISH ST.,<NA>,<NA>,35.995213,-78.899873,0.248682
1329,ACTIVE,"[35.9956634, -78.8998729]",27701,Food Service,4,<NA>,120.0,5 - Municipal/Community,(919) 596-2750,NC,...,FOOD,NO,70446,3 - Municipal/Community,200 N. MANGUM ST. SUITE 101,<NA>,<NA>,35.995663,-78.899873,0.260197
898,ACTIVE,"[35.9970933, -78.8992279]",27701,Food Service,4,<NA>,90.0,5 - Municipal/Community,(919) 251-9335,NC,...,FOOD,N/A,162302,3 - Municipal/Community,415 E CHAPEL HILL STREET,<NA>,<NA>,35.997093,-78.899228,0.306987
2038,ACTIVE,"[35.9946635, -78.9007481]",27701,Food Service,2,Full-Service Restaurant,49.0,5 - Municipal/Community,(919) 973-3000,NC,...,FOOD,N/A,161631,3 - Municipal/Community,105 S MANGUM STRRET SUITE 100,<NA>,<NA>,35.994664,-78.900748,0.326422


### (b) With that restautant in (a) as your center point, find the number of foreclosures within a 1 mile radius.

In [13]:
coord2 = (result1.compute().iloc[0]['latitude'], result1.compute().iloc[0]['longitude'])
print("Restaurant:", result1.compute().iloc[0]['premise_name'], coord2)

Restaurant: OLD HAVANA SANDWICH SHOP (np.float64(35.9932826), np.float64(-78.8981331))


In [14]:
file_path = "shared/data/durham-nc-foreclosure-2006-2016.json"
df2 = pd.read_json(file_path)

In [15]:
df2_fields = pd.json_normalize(df2['fields'])
df2 = pd.concat([df2.drop(columns=['fields']), df2_fields], axis=1)

In [16]:
df2['latitude'] = df2['geocode'].apply(lambda x: x[0] if isinstance(x, list) else None)
df2['longitude'] = df2['geocode'].apply(lambda x: x[1] if isinstance(x, list) else None)

In [17]:
dask_df2 = dd.from_pandas(df2, npartitions=4)

In [18]:
dask_df2

,datasetid,recordid,geometry,record_timestamp,parcel_number,geocode,address,year,latitude,longitude
npartitions=4,,,,,,,,,,
0,string,string,string,string,string,string,string,string,float64,float64
487,...,...,...,...,...,...,...,...,...,...
974,...,...,...,...,...,...,...,...,...,...
1461,...,...,...,...,...,...,...,...,...,...
1947,...,...,...,...,...,...,...,...,...,...


In [19]:
def calculate_within_radius(df, p):
    df['within_radius'] = df.apply(
        lambda row: haversine((row['latitude'], row['longitude']), p, unit=Unit.MILES) <= 1.0,
        axis=1
    )
    return df

In [20]:
dask_df2 = dask_df2.map_partitions(calculate_within_radius, coord2)

In [21]:
dask_df2 = dask_df2[dask_df2['within_radius'] == True]

In [22]:
print("The number of foreclosures within a 1 mile radius:", dask_df2.compute().shape[0])

The number of foreclosures within a 1 mile radius: 320


## 2 Same as HW1 Q2.1, Language Models, in Dask - 50 points

Input: hw1text.zip (provided in class website)   
Solve: Compute the top 3 most likely words following "new".

Hint: P(w2|w1) = conditional probability of w2 given w1.

In [23]:
import zipfile
import os
import re
import dask.bag as db

In [24]:
file_path = "hw1text.zip"
directory = "hw1text"
os.makedirs(directory, exist_ok=True)

with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(directory)

files = os.listdir(directory)
files

['20-04.txt', '20-03.txt', '20-05.txt', '20-01.txt', '20-02.txt']

In [25]:
text_files = [
    f"{directory}/20-01.txt",
    f"{directory}/20-02.txt",
    f"{directory}/20-03.txt",
    f"{directory}/20-04.txt",
    f"{directory}/20-05.txt",
]

In [26]:
df3 = db.read_text(text_files)

In [27]:
# Tokenize and clean the text
def norm(text):
    cleaned_text = re.sub(r'[^0-9a-z]', ' ', text.lower())
    return cleaned_text.split()

In [28]:
df3 = df3.map(norm).flatten()

In [29]:
df3 = df3.frequencies()
word_count_df3 = df3.to_dataframe(columns=["word", "count"])
word_count_df3 = word_count_df3.sort_values(by="count", ascending=False)

In [30]:
word_count_df3.compute()

,word,count
2,the,163547
43,to,89046
1,p,78664
13,of,75568
82,and,72730
...,...,...
68085,85043941,1
68087,tamika,1
68088,catchings,1
68072,bmorethical,1


In [31]:
word_count_with_new = word_count_df3[word_count_df3["word"] == "new"]
word_count_with_new.compute()

,word,count
433,new,6832


In [32]:
def compute_bigrams(words):
    return [(words[i], words[i + 1]) for i in range(len(words) - 1)]

In [33]:
df3 = db.read_text(text_files)
df3 = df3.map(norm).map(compute_bigrams).flatten()

In [34]:
df3 = df3.frequencies()
bigram_count_df3 = df3.to_dataframe(columns=["bigram", "count"])
bigram_count_df3 = bigram_count_df3.sort_values(by="count", ascending=False)

In [35]:
bigram_count_with_new = bigram_count_df3[bigram_count_df3["bigram"].str.startswith("('new', ")]
bigram_count_with_new.compute()

,bigram,count
6542,"('new', 'york')",910
5425,"('new', 'cases')",460
1286,"('new', 'zealand')",424
2456,"('new', 'coronavirus')",395
6550,"('new', 'jersey')",234
...,...,...
848724,"('new', 'retail')",1
848753,"('new', 'by')",1
848814,"('new', 'though')",1
848841,"('new', 'does')",1


In [36]:
new_count = word_count_with_new.compute()["count"].values[0]
print(new_count)

6832


In [37]:
bigram_count_with_new["P(w2|w1)"] = bigram_count_with_new["count"] / new_count

In [38]:
print("Top-3 most likely words following 'new':")
bigram_count_with_new.compute().head(3)

Top-3 most likely words following 'new':


,bigram,count,P(w2|w1)
6542,"('new', 'york')",910,0.133197
5425,"('new', 'cases')",460,0.067330
1286,"('new', 'zealand')",424,0.062061


In [39]:
client.close()